In [1]:
from huggingface_hub import notebook_login
notebook_login()

In [2]:
from datasets import load_dataset, Audio

In [3]:
dataset = load_dataset("audiofolder", data_dir=r"/home/intern/subham/Lang_data", split="train")
dataset = dataset.cast_column("audio", Audio(sampling_rate=16000))

Resolving data files:   0%|          | 0/22887 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [4]:
dataset

Dataset({
    features: ['audio', 'label'],
    num_rows: 22887
})

In [5]:
# Split first
import numpy as np
minds = dataset.train_test_split(test_size=0.2)

# Convert label from str to int
minds = minds.map(lambda x: {"label":  int(x["label"])})





Map:   0%|          | 0/18309 [00:00<?, ? examples/s]

Map:   0%|          | 0/4578 [00:00<?, ? examples/s]

In [6]:
print(minds['train'].features)



{'audio': Audio(sampling_rate=16000, mono=True, decode=True, id=None), 'label': ClassLabel(names=['Kabui', 'Manipuri', 'Mao', 'Poumei', 'Tangkhul', 'Thadou'], id=None)}


In [7]:
import librosa
import soundfile as sf

minds["train"][0]

{'audio': {'path': '/home/intern/subham/Lang_data/Poumei/mai-poum-mess_136.wav',
  'array': array([-0.04292756, -0.07249045, -0.06649353, ...,  0.00229349,
          0.00603162,  0.00997048], shape=(479915,)),
  'sampling_rate': 16000},
 'label': 3}

In [8]:
labels = dataset.features["label"].names  # <- directory names in alphabetical order
label2id = {label: i for i, label in enumerate(labels)}
id2label = {i: label for label, i in label2id.items()}

print("label2id :", label2id)
print("id2label :", id2label)


label2id : {'Kabui': 0, 'Manipuri': 1, 'Mao': 2, 'Poumei': 3, 'Tangkhul': 4, 'Thadou': 5}
id2label : {0: 'Kabui', 1: 'Manipuri', 2: 'Mao', 3: 'Poumei', 4: 'Tangkhul', 5: 'Thadou'}


In [9]:
from transformers import AutoFeatureExtractor

feature_extractor = AutoFeatureExtractor.from_pretrained("fydhfzh/wav2vec2-classifier-aug")

In [10]:
minds = minds.cast_column("audio", Audio(sampling_rate=16_000))
minds["train"][0]

{'audio': {'path': '/home/intern/subham/Lang_data/Poumei/mai-poum-mess_136.wav',
  'array': array([-0.04292756, -0.07249045, -0.06649353, ...,  0.00229349,
          0.00603162,  0.00997048], shape=(479915,)),
  'sampling_rate': 16000},
 'label': 3}

In [11]:
def preprocess_function(examples):
    audio_arrays = [x["array"] for x in examples["audio"]]
    inputs = feature_extractor(
        audio_arrays, sampling_rate=feature_extractor.sampling_rate, max_length=16000*15, truncation=True
    )
    return inputs

In [12]:
import datasets


encoded_minds = minds.map(preprocess_function, remove_columns="audio", batched=True)
encoded_minds = encoded_minds.cast_column("label", datasets.Value("int64"))
encoded_minds = encoded_minds.with_format("torch")


Map:   0%|          | 0/18309 [00:00<?, ? examples/s]

Map:   0%|          | 0/4578 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/18309 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/4578 [00:00<?, ? examples/s]

In [13]:
from transformers.modeling_utils import PreTrainedModel
from transformers import AutoModelForAudioClassification, TrainingArguments, Trainer

num_labels = len(id2label)
model = AutoModelForAudioClassification.from_pretrained(
    "fydhfzh/wav2vec2-classifier-aug",
    num_labels=num_labels ,
    label2id=label2id,
    id2label=id2label,
    ignore_mismatched_sizes=True
)

Some weights of Wav2Vec2ForSequenceClassification were not initialized from the model checkpoint at fydhfzh/wav2vec2-classifier-aug and are newly initialized because the shapes did not match:
- classifier.bias: found shape torch.Size([84]) in the checkpoint and torch.Size([6]) in the model instantiated
- classifier.weight: found shape torch.Size([84, 256]) in the checkpoint and torch.Size([6, 256]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [14]:
from transformers import DataCollatorWithPadding

class MyDataCollator(DataCollatorWithPadding):
    """Custom collator to make sure labels are LongTensor."""
    def __call__(self, features):
        batch = super().__call__(features)
        batch["labels"] = batch["labels"].long()
        return batch

data_collator = MyDataCollator(feature_extractor)


In [ ]:
from transformers import TrainingArguments, Trainer, set_seed
from transformers.utils import logging
import evaluate

# Optional: Set seed and enable logging info
set_seed(42)
logging.set_verbosity_info()

# FIX compute_metrics first
import numpy as np

accuracy = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred.predictions, eval_pred.label_ids
    preds = np.argmax(logits, axis=-1)
    return {"accuracy": accuracy.compute(predictions=preds, references=labels)['accuracy']}

training_args = TrainingArguments(
    output_dir="my_awesome_mind_model",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=3e-5,  # slightly lower LR
    per_device_train_batch_size=16,  # double the batch size
    per_device_eval_batch_size=16,
    gradient_accumulation_steps=2,  # cut back accumulation
    num_train_epochs=1,  # train for a bit longer
    warmup_ratio=0.05,  # small warm-up
    weight_decay=0.01,  # a bit of weight decay
    logging_steps=100,
    logging_dir="./logs",
    report_to="tensorboard",
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    push_to_hub=False,
   
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=encoded_minds["train"],
    eval_dataset=encoded_minds["test"],
    tokenizer=feature_extractor,         # Use tokenizer or feature_extractor
    compute_metrics=compute_metrics
)






trainer.train()

PyTorch: setting up devices
/tmp/ipykernel_1366696/3096157639.py:39: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
***** Running training *****
  Num examples = 18,309
  Num Epochs = 1
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 2
  Total optimization steps = 573
  Number of trainable parameters = 94,570,118


Epoch,Training Loss,Validation Loss


In [ ]:
# Saves the trained model weights and configuration
model.save_pretrained("./my_trained_model")

# Saves the corresponding tokenizer or feature extractor
feature_extractor.save_pretrained("./my_trained_model")


Configuration saved in ./my_trained_model/config.json
Model weights saved in ./my_trained_model/model.safetensors
Feature extractor saved in ./my_trained_model/preprocessor_config.json


['./my_trained_model/preprocessor_config.json']

In [7]:
import torch
import librosa



# 1️⃣ Prepare the audio first
audio_file = r"/home/intern/subham/mni-mao/mni-mao000.wav" 
audio, sr = librosa.load(audio_file, sr=16000)

inputs = feature_extractor(audio, sampling_rate=16000, max_length=16000*5,return_tensors='pt')

# 2️⃣ Send both model and inputs to the same device
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)
inputs = {key: val.to(device) for key, val in inputs.items()}

# 3️⃣ Forward pass
with torch.no_grad():
    logits = model(**inputs).logits

# 4️⃣ Prediction
predicted_class = logits.argmax(-1).item()
label = model.config.id2label[predicted_class]

print("Predicted class :", label)

NameError: name 'feature_extractor' is not defined

In [19]:
#yt-dlp -x --audio-format wav --ffmpeg-location "/home/intern/subham/.conda/bin" "https://www.youtube.com/watch?v=fblSGnBv9CI"
#  cd /home/intern/subham/.conda/lib/
# ln -s libopenh264.so.6 libopenh264.so.5
# ffmpeg -i "/home/intern/subham/Story Of Ningthem Pukhri [xI8meVJL2aA].webm" -ac 2 -ar 16000 output.wav
#ffmpeg -i /home/intern/subham/manipuri/Storyoutput.wav -f segment -segment_time 30 -c copy output_%03d.wav

In [ ]:
# Array.from(document.querySelectorAll('a#video-title')).map(el => el.href); js code 

In [ ]:
# # link_arr_tangkul=['https://www.youtube.com/watch?v=fblSGnBv9CI&t=29s&pp=ygUNVEFOR0tIVUwgTkVXUw%3D%3D', 'https://www.youtube.com/watch?v=H5E_IPM6pVs&t=159s&pp=ygUNVEFOR0tIVUwgTkVXUw%3D%3D', 'https://www.youtube.com/watch?v=j5FE91XcFBE&pp=ygUNVEFOR0tIVUwgTkVXUw%3D%3D', 'https://www.youtube.com/watch?v=DyL8s4Vmukg&pp=ygUNVEFOR0tIVUwgTkVXUw%3D%3D', 'https://www.youtube.com/watch?v=ChPbO4iHsj4&pp=ygUNVEFOR0tIVUwgTkVXUw%3D%3D', 'https://www.youtube.com/watch?v=AYU3tu5ra6I&pp=ygUNVEFOR0tIVUwgTkVXUw%3D%3D', 'https://www.youtube.com/watch?v=oSPXt-qVQQM&pp=ygUNVEFOR0tIVUwgTkVXUw%3D%3D', 'https://www.youtube.com/watch?v=6feFd6pX6z4&pp=ygUNVEFOR0tIVUwgTkVXUw%3D%3D', 'https://www.youtube.com/watch?v=q1wYC98KcCQ&pp=ygUNVEFOR0tIVUwgTkVXUw%3D%3D', 'https://www.youtube.com/watch?v=jLo9RXl0_Dw&pp=ygUNVEFOR0tIVUwgTkVXUw%3D%3D', 'https://www.youtube.com/watch?v=9kQOhIZrZDs&pp=ygUNVEFOR0tIVUwgTkVXUw%3D%3D', 'https://www.youtube.com/watch?v=-ILfpFT_v1o&pp=ygUNVEFOR0tIVUwgTkVXUw%3D%3D', 'https://www.youtube.com/watch?v=IXrtFT_WYrg&pp=ygUNVEFOR0tIVUwgTkVXUw%3D%3D', 'https://www.youtube.com/watch?v=fZs92G4FYQQ&pp=ygUNVEFOR0tIVUwgTkVXUw%3D%3D', 'https://www.youtube.com/watch?v=0UKmLTlkjhw&pp=ygUNVEFOR0tIVUwgTkVXUw%3D%3D', 'https://www.youtube.com/watch?v=Z3rqoGz-PRk&pp=ygUNVEFOR0tIVUwgTkVXUw%3D%3D', 'https://www.youtube.com/watch?v=XJySImUzQbY&pp=ygUNVEFOR0tIVUwgTkVXUw%3D%3D', 'https://www.youtube.com/watch?v=Lw7Oa1-WjUk&pp=ygUNVEFOR0tIVUwgTkVXU9IHCQneCQGHKiGM7w%3D%3D', 'https://www.youtube.com/watch?v=Fvvwl_2CNUE&pp=ygUNVEFOR0tIVUwgTkVXUw%3D%3D', 'https://www.youtube.com/watch?v=eOdJJFcTOhM&pp=ygUNVEFOR0tIVUwgTkVXUw%3D%3D']
# #done

# #manipuri
# arr= [
#     "https://youtu.be/KoSsemIIat4?si=hWhhT21xFJE8xko5",
#     "https://youtu.be/3oxXjQZpw8Y?si=XgGjxhOfECnTAS9Q",
#     "https://youtu.be/FtUYJ4iPQmw?si=RCDE945nEzp4cVTi",
#     "https://youtu.be/2XgfjuyD2bs?si=EBEmwyJoXAwdFXH4",
#     "https://youtu.be/aSyhbPYXQmU?si=Apyd8QNYQ1nkNICC",
#     "https://youtu.be/1UZ5uCGetzA?si=wxlCk5wOX7srOXNQ",
#     "https://youtu.be/HBa_bO97nNs?si=U8Ua3EydAEwBbp6a",
#     "https://youtu.be/tt7QopS_DIM?si=r9-8X12Pb4iR_TZ7",
#     "https://youtu.be/x0bxvCkjWbc?si=19opHR-1Y8dX9VDB",
#     "https://youtu.be/8ONNDOJcyHQ?si=UKIaG07Iom4IPo6U",
#     "https://youtu.be/OO0AgP4sgCE?si=IBWAgNsOn4c4qSAc",
#     "https://youtu.be/1C6O4Q4Ygbs?si=7-pLxSUF7RL6uArF",
#     "https://youtu.be/BrfLnWLci1M?si=t5tMO-v5xjhDxCBT",
#     "https://youtu.be/Sj_4X2P8tAU?si=eEHCAIVnDwhypQmB",
#     "https://youtu.be/kaE9VAUN8R8?si=h-8lLo4djjddiKUX"
# ]



# foldername="Mao"
# counter=0
# !mkdir -p "{foldername}"
# for i in arr:
#         !yt-dlp -x --audio-format wav --ffmpeg-location "/home/intern/subham/.conda/bin" -o "{foldername}/{foldername}00{counter}.wav" "{i}"
#         counter+=1


[youtube] Extracting URL: https://youtu.be/KoSsemIIat4?si=hWhhT21xFJE8xko5
[youtube] KoSsemIIat4: Downloading webpage
[youtube] KoSsemIIat4: Downloading tv client config
[youtube] KoSsemIIat4: Downloading tv player API JSON
[youtube] KoSsemIIat4: Downloading ios player API JSON
[youtube] KoSsemIIat4: Downloading m3u8 information
[info] KoSsemIIat4: Downloading 1 format(s): 251
[download] Destination: Mao/Mao000.webm
[download] 100% of   53.40MiB in 00:00:05 at 9.79MiB/s0;33m00:000m
[ExtractAudio] Destination: Mao/Mao000.wav
Deleting original file Mao/Mao000.webm (pass -k to keep)
[youtube] Extracting URL: https://youtu.be/3oxXjQZpw8Y?si=XgGjxhOfECnTAS9Q
[youtube] 3oxXjQZpw8Y: Downloading webpage
[youtube] 3oxXjQZpw8Y: Downloading tv client config
[youtube] 3oxXjQZpw8Y: Downloading tv player API JSON
[youtube] 3oxXjQZpw8Y: Downloading ios player API JSON
[youtube] 3oxXjQZpw8Y: Downloading m3u8 information
[info] 3oxXjQZpw8Y: Downloading 1 format(s): 328
[download] Destination: Mao/Mao00

In [ ]:
# %%bash
# DIRNAME="Mao"

# folder="/home/intern/subham/${DIRNAME}"
# subfolder="mni-mao-mess"
# mkdir -p "$folder/$subfolder"

# for file in "$folder"/*.wav; do
#     original_name=$(basename "$file" .wav)  # strip .wav extension
#     output_pattern="$folder/$subfolder/${original_name}_%03d.wav"

#     ffmpeg -i "$file" -f segment -segment_time 30 -c copy "$output_pattern"

#     echo "Done segmenting $file into ${original_name}_***.wav"
# done


ffmpeg version 4.3 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 7.3.0 (crosstool-NG 1.23.0.449-a04d0)
  configuration: --prefix=/home/intern/subham/.conda --cc=/opt/conda/conda-bld/ffmpeg_1597178665428/_build_env/bin/x86_64-conda_cos6-linux-gnu-cc --disable-doc --disable-openssl --enable-avresample --enable-gnutls --enable-hardcoded-tables --enable-libfreetype --enable-libopenh264 --enable-pic --enable-pthreads --enable-shared --disable-static --enable-version3 --enable-zlib --enable-libmp3lame
  libavutil      56. 51.100 / 56. 51.100
  libavcodec     58. 91.100 / 58. 91.100
  libavformat    58. 45.100 / 58. 45.100
  libavdevice    58. 10.100 / 58. 10.100
  libavfilter     7. 85.100 /  7. 85.100
  libavresample   4.  0.  0 /  4.  0.  0
  libswscale      5.  7.100 /  5.  7.100
  libswresample   3.  7.100 /  3.  7.100


Guessed Channel Layout for Input Stream #0.0 : stereo
Input #0, wav, from '/home/intern/subham/Mao/Mao000.wav':
  Metadata:
    encoder         : Lavf58.45.100
  Duration: 01:11:40.12, bitrate: 1536 kb/s
    Stream #0:0: Audio: pcm_s16le ([1][0][0][0] / 0x0001), 48000 Hz, stereo, s16, 1536 kb/s
[segment @ 0x619fe9a19400] Opening '/home/intern/subham/Mao/mni-mao-mess/Mao000_000.wav' for writing
Output #0, segment, to '/home/intern/subham/Mao/mni-mao-mess/Mao000_%03d.wav':
  Metadata:
    encoder         : Lavf58.45.100
    Stream #0:0: Audio: pcm_s16le ([1][0][0][0] / 0x0001), 48000 Hz, stereo, s16, 1536 kb/s
Stream mapping:
  Stream #0:0 -> #0:0 (copy)
Press [q] to stop, [?] for help
[segment @ 0x619fe9a19400] Opening '/home/intern/subham/Mao/mni-mao-mess/Mao000_001.wav' for writing
[segment @ 0x619fe9a19400] Opening '/home/intern/subham/Mao/mni-mao-mess/Mao000_002.wav' for writing
[segment @ 0x619fe9a19400] Opening '/home/intern/subham/Mao/mni-mao-mess/Mao000_003.wav' for writing
[seg

Done segmenting /home/intern/subham/Mao/Mao000.wav into Mao000_***.wav


ffmpeg version 4.3 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 7.3.0 (crosstool-NG 1.23.0.449-a04d0)
  configuration: --prefix=/home/intern/subham/.conda --cc=/opt/conda/conda-bld/ffmpeg_1597178665428/_build_env/bin/x86_64-conda_cos6-linux-gnu-cc --disable-doc --disable-openssl --enable-avresample --enable-gnutls --enable-hardcoded-tables --enable-libfreetype --enable-libopenh264 --enable-pic --enable-pthreads --enable-shared --disable-static --enable-version3 --enable-zlib --enable-libmp3lame
  libavutil      56. 51.100 / 56. 51.100
  libavcodec     58. 91.100 / 58. 91.100
  libavformat    58. 45.100 / 58. 45.100
  libavdevice    58. 10.100 / 58. 10.100
  libavfilter     7. 85.100 /  7. 85.100
  libavresample   4.  0.  0 /  4.  0.  0
  libswscale      5.  7.100 /  5.  7.100
  libswresample   3.  7.100 /  3.  7.100
Guessed Channel Layout for Input Stream #0.0 : stereo
Input #0, wav, from '/home/intern/subham/Mao/Mao0010.wav':
  Metadata:
    encoder         : Lavf58.

Done segmenting /home/intern/subham/Mao/Mao0010.wav into Mao0010_***.wav


...

Done segmenting /home/intern/subham/Mao/Mao007.wav into Mao007_***.wav


ffmpeg version 4.3 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 7.3.0 (crosstool-NG 1.23.0.449-a04d0)
  configuration: --prefix=/home/intern/subham/.conda --cc=/opt/conda/conda-bld/ffmpeg_1597178665428/_build_env/bin/x86_64-conda_cos6-linux-gnu-cc --disable-doc --disable-openssl --enable-avresample --enable-gnutls --enable-hardcoded-tables --enable-libfreetype --enable-libopenh264 --enable-pic --enable-pthreads --enable-shared --disable-static --enable-version3 --enable-zlib --enable-libmp3lame
  libavutil      56. 51.100 / 56. 51.100
  libavcodec     58. 91.100 / 58. 91.100
  libavformat    58. 45.100 / 58. 45.100
  libavdevice    58. 10.100 / 58. 10.100
  libavfilter     7. 85.100 /  7. 85.100
  libavresample   4.  0.  0 /  4.  0.  0
  libswscale      5.  7.100 /  5.  7.100
  libswresample   3.  7.100 /  3.  7.100
Guessed Channel Layout for Input Stream #0.0 : stereo
Input #0, wav, from '/home/intern/subham/Mao/Mao008.wav':
  Metadata:
    encoder         : Lavf58.4

Done segmenting /home/intern/subham/Mao/Mao008.wav into Mao008_***.wav


ffmpeg version 4.3 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 7.3.0 (crosstool-NG 1.23.0.449-a04d0)
  configuration: --prefix=/home/intern/subham/.conda --cc=/opt/conda/conda-bld/ffmpeg_1597178665428/_build_env/bin/x86_64-conda_cos6-linux-gnu-cc --disable-doc --disable-openssl --enable-avresample --enable-gnutls --enable-hardcoded-tables --enable-libfreetype --enable-libopenh264 --enable-pic --enable-pthreads --enable-shared --disable-static --enable-version3 --enable-zlib --enable-libmp3lame
  libavutil      56. 51.100 / 56. 51.100
  libavcodec     58. 91.100 / 58. 91.100
  libavformat    58. 45.100 / 58. 45.100
  libavdevice    58. 10.100 / 58. 10.100
  libavfilter     7. 85.100 /  7. 85.100
  libavresample   4.  0.  0 /  4.  0.  0
  libswscale      5.  7.100 /  5.  7.100
  libswresample   3.  7.100 /  3.  7.100
Guessed Channel Layout for Input Stream #0.0 : stereo
Input #0, wav, from '/home/intern/subham/Mao/Mao009.wav':
  Metadata:
    encoder         : Lavf58.4

Done segmenting /home/intern/subham/Mao/Mao009.wav into Mao009_***.wav


In [ ]:
# %%bash
# parent_folder="/home/intern/subham/Tangkhul"

# # Loop through each subfolder
# for subfolder in "$parent_folder"/*/; do
#     # Get subfolder name only (basename)
#     subfolder_name=$(basename "$subfolder")
    
#     counter=0

#     # Loop through all wav files inside the subfolder
#     for file in "$subfolder"*.wav; do
#         [ -e "$file" ] || continue  # Skip if no .wav files
        
#         original_name=$(basename "$file")
#         new_name=$(printf "%s_%02d.wav" "$subfolder_name" "$counter")
        
#         mv "$file" "$subfolder/$new_name"
#         echo "Renamed $original_name to $new_name in $subfolder_name"
        
#         counter=$((counter + 1))
#     done
# done


Renamed Tangkhul000_000.wav to mni-tang-mess_00.wav in mni-tang-mess
Renamed Tangkhul000_001.wav to mni-tang-mess_01.wav in mni-tang-mess
Renamed Tangkhul000_002.wav to mni-tang-mess_02.wav in mni-tang-mess
Renamed Tangkhul000_003.wav to mni-tang-mess_03.wav in mni-tang-mess
Renamed Tangkhul000_004.wav to mni-tang-mess_04.wav in mni-tang-mess
Renamed Tangkhul000_005.wav to mni-tang-mess_05.wav in mni-tang-mess
Renamed Tangkhul000_006.wav to mni-tang-mess_06.wav in mni-tang-mess
Renamed Tangkhul000_007.wav to mni-tang-mess_07.wav in mni-tang-mess


Renamed Tangkhul000_008.wav to mni-tang-mess_08.wav in mni-tang-mess
Renamed Tangkhul000_009.wav to mni-tang-mess_09.wav in mni-tang-mess
Renamed Tangkhul000_010.wav to mni-tang-mess_10.wav in mni-tang-mess
Renamed Tangkhul000_011.wav to mni-tang-mess_11.wav in mni-tang-mess
Renamed Tangkhul000_012.wav to mni-tang-mess_12.wav in mni-tang-mess
Renamed Tangkhul000_013.wav to mni-tang-mess_13.wav in mni-tang-mess
Renamed Tangkhul000_014.wav to mni-tang-mess_14.wav in mni-tang-mess
Renamed Tangkhul000_015.wav to mni-tang-mess_15.wav in mni-tang-mess
Renamed Tangkhul000_016.wav to mni-tang-mess_16.wav in mni-tang-mess
Renamed Tangkhul000_017.wav to mni-tang-mess_17.wav in mni-tang-mess
Renamed Tangkhul000_018.wav to mni-tang-mess_18.wav in mni-tang-mess
Renamed Tangkhul000_019.wav to mni-tang-mess_19.wav in mni-tang-mess
Renamed Tangkhul000_020.wav to mni-tang-mess_20.wav in mni-tang-mess
Renamed Tangkhul000_021.wav to mni-tang-mess_21.wav in mni-tang-mess
Renamed Tangkhul000_022.wav to mni

In [ ]:
# %%bash
# parent_folder="/home/intern/subham/Tangkhul"
# target_folder="/home/intern/subham/Lang_data/Tangkhul"

# mkdir -p "$target_folder"

# # Loop through each subfolder
# for subfolder in "$parent_folder"/*/; do
#     subfolder_name=$(basename "$subfolder")
#     counter=0

#     # Loop through all .wav files inside the subfolder
#     for file in "$subfolder"*.wav; do
#         [ -e "$file" ] || continue  # Skip if no files
        
#         original_name=$(basename "$file")
#         new_name=$(printf "%s_%02d.wav" "$subfolder_name" "$counter")
        
#         mv "$file" "$target_folder/$new_name"
#         echo "Moved and renamed $original_name to $new_name"
        
#         counter=$((counter + 1))
#     done
# done


Moved and renamed mni-tang-mess_00.wav to mni-tang-mess_00.wav


Moved and renamed mni-tang-mess_01.wav to mni-tang-mess_01.wav
Moved and renamed mni-tang-mess_02.wav to mni-tang-mess_02.wav
Moved and renamed mni-tang-mess_03.wav to mni-tang-mess_03.wav
Moved and renamed mni-tang-mess_04.wav to mni-tang-mess_04.wav
Moved and renamed mni-tang-mess_05.wav to mni-tang-mess_05.wav
Moved and renamed mni-tang-mess_06.wav to mni-tang-mess_06.wav
Moved and renamed mni-tang-mess_07.wav to mni-tang-mess_07.wav
Moved and renamed mni-tang-mess_08.wav to mni-tang-mess_08.wav
Moved and renamed mni-tang-mess_09.wav to mni-tang-mess_09.wav
Moved and renamed mni-tang-mess_1000.wav to mni-tang-mess_10.wav
Moved and renamed mni-tang-mess_1001.wav to mni-tang-mess_11.wav
Moved and renamed mni-tang-mess_1002.wav to mni-tang-mess_12.wav
Moved and renamed mni-tang-mess_1003.wav to mni-tang-mess_13.wav
Moved and renamed mni-tang-mess_1004.wav to mni-tang-mess_14.wav
Moved and renamed mni-tang-mess_1005.wav to mni-tang-mess_15.wav
Moved and renamed mni-tang-mess_1006.wav to